In [ ]:
!pip install foolbox

     |████████████████████████████████| 1.7MB 8.3MB/s 
     |████████████████████████████████| 163kB 48.2MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD,LinfBasicIterativeAttack,LinfFastGradientAttack,L2CarliniWagnerAttack,LinfDeepFoolAttack,L2DeepFoolAttack

In [ ]:
from __future__ import print_function
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import warnings
import torch.optim as optim
import torch.nn as nn
import sys
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from datetime import datetime
from numpy.linalg import norm
import torchvision.models as models

In [ ]:
class CNN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        """Perform forward."""

        # conv layers
        x = self.conv_layer(x)

        # flatten
        x = x.view(x.size(0), -1)

        # fc layer
        x = self.fc_layer(x)

        return x

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_cnn = CNN()
model_cnn.load_state_dict(torch.load("model_cnn_cifar_new.pt", map_location=device))
model_cnn = model_cnn.to(device)
model_cnn.eval()

softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

corrects = []
corrects_tuple_list = []

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

2021-02-24 12:08:58


In [ ]:
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None,:,None,None]) / self.std.type_as(x)[None,:,None,None]

batch_size = 64
norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform = transforms.ToTensor()

test_data = datasets.CIFAR10(root='/CIFARDATA', train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Extracting /CIFARDATA/cifar-10-python.tar.gz to /CIFARDATA


In [ ]:
warnings.filterwarnings("ignore")


torch.manual_seed(2)
np.random.seed(2)

eps = 2/255
alpha = 0.2 * eps
num_iter = 10

eps_little = 0.01/255
alpha_reverse = 0.2 * eps_little
num_iter_reverse = 10

count_successful_reverse = 0
count_unsuccessful_reverse = 0

count_successfull_attack = 0
count_unsuccessfull_attack = 0

In [ ]:
def fgsm(image,model,epsilon, label):

    image = image.detach()

    item_count = image.shape[0]

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()
    o = model(norm((image + delta).clamp(0, 1)))
    loss = nn.CrossEntropyLoss(reduce=False)(o, label)
    loss = loss.reshape(1, item_count)
    

    loss.backward(torch.ones_like(loss))

    delta.data = delta + epsilon * delta.grad.detach().sign()
    delta.grad.zero_()


    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [ ]:
def bim(image,model,epsilon, label, alpha, num_iter):

    image = image.detach()

    item_count = image.shape[0]

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None


    for t in range(num_iter):

        model.eval()
        o = model(norm((image + delta).clamp(0, 1)))

        loss = nn.CrossEntropyLoss(reduce=False)(o, label)
        loss = loss.reshape(1, item_count)

        loss.backward(torch.ones_like(loss))

        delta.data = (delta + alpha * delta.grad.detach().sign()).clamp(-epsilon, epsilon)
        delta.grad.zero_()


    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [ ]:

def rectified_bim(image,model,epsilon, num_iter, alpha):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(norm(image))
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([50,item_count,10])

        for i in range(50):

            enable_dropout(model)
            output = model(norm((image + delta).clamp(0, 1)))
            output = softmax(output)

            dropout_predictions[i] = output


        variance = torch.var(dropout_predictions, dim=0)

        var = variance.mean(1,True)
        var = var.reshape(1,item_count)
        var = var.to(device)


        model.eval()
        o = model(norm((image + delta).clamp(0, 1)))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
            delta.data = (delta + alpha * delta_unc).clamp(-epsilon, epsilon)


        else:

            model.eval()
            with torch.no_grad():
                o = model(norm((image + delta).clamp(0, 1)))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

##########################################################################################

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp, retain_graph=True)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp, retain_graph=True)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None


            zeros = torch.zeros_like(delta_loss)
            delta_loss = torch.where(delta_unc != delta_loss, zeros, delta_loss)
            delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)


##########################################################################################

            inds_match = np.where(inter_pred.cpu() == init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_match] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_loss = torch.where(delta_unc == delta_loss, zeros, delta_loss)
            delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)

##########################################################################################

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [ ]:

for i, (image, label) in enumerate(test_loader):

    if i%30==0:
      print(i)

    image = image.to(device)
    label = label.to(device)

    model_cnn.eval()

    with torch.no_grad():
        o = model_cnn(norm(image))
        o = softmax(o)

    pred_original = o.data.max(1, keepdim=True)[1]
    pred_original = pred_original.view_as(label)
    inds_correct = np.where(pred_original.cpu() == label.cpu())[0]

    image = image[inds_correct]
    label = label[inds_correct]


    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)

    #pert = fgsm(image, model_cnn, eps, label)

    #pert = bim(image, model_cnn, eps, label, alpha, num_iter)

    pert = rectified_bim(image, model_cnn, eps, num_iter, alpha)

    # attack = L2DeepFoolAttack()
    # fmodel = PyTorchModel(model_cnn, bounds=(0, 1),preprocessing=preprocessing)
    # raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[1.35])
    # pert = torch.tensor(clipped_advs[0])


    #attack = LinfDeepFoolAttack()
    #fmodel = PyTorchModel(model_cnn, bounds=(0, 1),preprocessing=preprocessing)
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps])
    #pert = torch.tensor(clipped_advs[0])

    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(norm(pert))
        o = softmax(o)
    pred_pert = o.data.max(1, keepdim=True)[1]


    pred_pert = pred_pert.view_as(label)


    inds_correct_after_attack = np.where(pred_pert.cpu() == label.cpu())[0]
    inds_wrong_after_attack = np.where(pred_pert.cpu() != label.cpu())[0]

    inds_correct_after_attackk = inds_correct_after_attack.tolist()
    inds_wrong_after_attackk = inds_wrong_after_attack.tolist()

    count_unsuccessfull_attack += len(inds_correct_after_attackk)
    count_successfull_attack += len(inds_wrong_after_attackk)

    image = image[inds_wrong_after_attack]
    label = label[inds_wrong_after_attack]
    pert = pert[inds_wrong_after_attack]

    #if i == 0:
        #break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
151
152
153
154
155
156


In [ ]:
print("Number of successful attack is : ", count_successfull_attack)
print("Number of unsuccessful attack is : ", count_unsuccessfull_attack)

print("Success ratio for the attack is : %{:.2f}".format(
    100 * count_successfull_attack / (count_successfull_attack + count_unsuccessfull_attack)))

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

Number of successful attack is :  7798
Number of unsuccessful attack is :  914
Success ratio for the attack is : %89.51
2021-02-24 12:44:03
